In [1]:
import pandas as pd
import requests

In [2]:
user = "jieqiangt@gmail.com"
pw = "tyz3xjz-tum*MEZ5fnh"

In [3]:

BASE_URL = "https://developers.onemap.sg/"

def searchPostal(postal_code):

    endpoint = "commonapi/search"
    headers = {'cache-control': 'no-cache, max-age=0',
               'content-type': 'application/json'}
    params = {"searchVal": postal_code, "returnGeom": "N",
              "getAddrDetails": "Y", "pageNum": 1}

    result = requests.get(url=f"{BASE_URL}/{endpoint}",
                          params=params, headers=headers)
    
    return result

def formatSearchPostal(result, postal_code):
    
    result_list = result.json()['results']
    returned_address = pd.DataFrame.from_dict(result_list[0], orient="index").T
    returned_address['INPUT_POSTAL'] = postal_code
    returned_address['QUERY_STATUS'] = "SUCCESS"
    
    return returned_address


In [20]:
postal_inputs

,Zipcode,Detail Address,Unit/floor,Additional address information
0,681166,166A Teck Whye Crescent,#18-355\t,NaN
1,540254,254 Compassvale Road,10-706\t,deliver 12pm
2,600273,273 Toh Guan Road,#02-55\t,NaN
3,531471,471A Upper Serangoon Crescent,#10-386\t,NaN
4,828746,"Waterwoods, 15 Punggol Field Walk",17-04\t,NaN
...,...,...,...,...
115,520298,Blk 298 Tampines St 22,#08-554\t,Call when reaching
116,427711,"Parc Seabreeze, 532 Joo Chiat Road",07-04\t,NaN
117,823268,268C Punggol Field,#03-159\t,NaN
118,438774,"Haig Residences, 163 Haig Road",#02-03\t,NaN


In [4]:

file="address_list.xlsx"
postal_inputs = pd.read_excel(file)

returned_addresses = []

for postal_code in postal_inputs['Zipcode']:
    print(postal_code)

    api_result = searchPostal(postal_code)

    if api_result.ok:
        result_dict = api_result.json()

    if len(result_dict["results"]) != 0:
        returned_address = formatSearchPostal(api_result, postal_code)
    else:
        returned_address = pd.DataFrame.from_dict({"BLK_NO": "", "ROAD_NAME": "", "BUILDING": "", "ADDRESS": "",
                                                  "POSTAL": "", "INPUT_POSTAL": postal_code, "QUERY_STATUS": "NO_RESULTS"}, orient="index").T
    
    returned_addresses.append(returned_address)


681166
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'TECK WHYE CREST', 'BLK_NO': '166A', 'ROAD_NAME': 'TECK WHYE CRESCENT', 'BUILDING': 'TECK WHYE CREST', 'ADDRESS': '166A TECK WHYE CRESCENT TECK WHYE CREST SINGAPORE 681166', 'POSTAL': '681166'}]}
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'TECK WHYE CREST', 'BLK_NO': '166A', 'ROAD_NAME': 'TECK WHYE CRESCENT', 'BUILDING': 'TECK WHYE CREST', 'ADDRESS': '166A TECK WHYE CRESCENT TECK WHYE CREST SINGAPORE 681166', 'POSTAL': '681166'}]}
540254
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'COMPASSVALE HAVEN', 'BLK_NO': '254', 'ROAD_NAME': 'COMPASSVALE ROAD', 'BUILDING': 'COMPASSVALE HAVEN', 'ADDRESS': '254 COMPASSVALE ROAD COMPASSVALE HAVEN SINGAPORE 540254', 'POSTAL': '540254'}]}
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'COMPASSVALE HAVEN', 'BLK_NO': '254', 'ROAD_NAME': 'COMPASSVALE ROAD', 'BUILDING': 'COMPASSVALE HAVEN', 'A

In [17]:
found_addresses = pd.concat(returned_addresses,ignore_index=True)
output = postal_inputs.merge(found_addresses, left_on="Zipcode", right_on='INPUT_POSTAL', how='left')
output.to_csv('returned_addresses.csv', index=False)


In [21]:
output[output['QUERY_STATUS'] != 'SUCCESS']

,Zipcode,Detail Address,Unit/floor,Additional address information,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,INPUT_POSTAL,QUERY_STATUS
84,409289,810 Geylang Road,04-27 City Plaza\t,NaN,NaN,,,,,,409289,NO_RESULTS


In [19]:
output[output['QUERY_STATUS'] != 'SUCCESS']

,Zipcode,Detail Address,Unit/floor,Additional address information,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,INPUT_POSTAL,QUERY_STATUS
84,409289,810 Geylang Road,04-27 City Plaza\t,NaN,NaN,,,,,,409289,NO_RESULTS


In [5]:
postal_code = 409289
test = searchPostal(postal_code)


In [6]:
test.json()

{'found': 0, 'totalNumPages': 0, 'pageNum': 1, 'results': []}

In [7]:
no_results = pd.DataFrame.from_dict({"BLK_NO": "","ROAD_NAME": "","BUILDING":"","ADDRESS":"","POSTAL":"", "INPUT_POSTAL":postal_code},orient="index").T
